In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import efficientnet.keras as efn 
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras import applications
from keras import callbacks
from keras.models import Sequential

from keras.layers import Dense
from keras.optimizers import Adam

In [ ]:
# Load the training data
train_df = pd.read_csv('/train_v2.csv')
train_df['image_name'] = train_df['image_name']+ '.jpg'
train_df

In [ ]:
# Fix tags column
train_df['tags'] = train_df['tags'].apply(lambda entry: entry.split(' '))

# Get a list of all unique labels
all_labels = set()
for tags in train_df['tags']:
    all_labels.update(tags)

# Create a dictionary to map labels to integers
label_to_int = {label: i for i, label in enumerate(all_labels)}
num_classes = len(label_to_int)

# Convert labels to one-hot encoded vectors
train_df['labels'] = train_df['tags'].apply(lambda entry: [label_to_int[label] for label in entry])

# Create separate columns for each label
for label in all_labels:
    train_df[label] = train_df['tags'].apply(lambda entry: int(label in entry))

train_df

In [ ]:
# Split the data into training and validation sets
train_set, val_set = train_test_split(train_df, test_size = 0.2, random_state = 42)
val_set

In [ ]:
# Define the image size and batch size
image_size = (256, 256)
batch_size = 8

In [ ]:
# Create an image data generator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=90)

In [ ]:
# Load and preprocess the training images
train_generator = train_datagen.flow_from_dataframe(
    dataframe = train_set,
    directory = '/kaggle/input/planet-understanding-the-amazon-from-space/train-jpg',
    x_col = 'image_name',
    y_col = list(all_labels),
    target_size = image_size,
    class_mode = 'raw',
    batch_size = batch_size,
    subset = 'training',
    interpolation = 'nearest')

train_generator

In [ ]:
# Load and preprocess the validation images
val_generator = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    directory = 'train-jpg/',
    x_col = 'image_name',
    y_col = list(all_labels),
    target_size = image_size,
    class_mode = 'raw',
    batch_size = 8,
    subset = 'training',
    interpolation = 'nearest')

val_generator

In [ ]:
efficient_net = efn.EfficientNetB3(
    weights='imagenet',
    input_shape=(256,256,3),
    include_top=False,
    pooling='max'
)

model = Sequential()
model.add(efficient_net)
model.add(Dense(units = 120, activation='relu'))
model.add(Dense(units = 140, activation = 'relu'))
model.add(Dense(units = 17, activation='sigmoid'))
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples//train_generator.batch_size,
    epochs = 70,
    validation_data=val_generator, validation_steps = val_generator.samples//val_generator.batch_size,
    verbose = 1)

In [ ]:
# Exporting happens here
model.save('model-ef')